In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/OptiML

In [ ]:
! pip install foolbox==3.3.1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, recall_score, f1_score
from Functions.implementations import *
from Functions.visualization import *
import foolbox as fb
from tqdm import tqdm
import os

In [46]:
def attack_n_batches(model, test_loader, device, n_batches=3):
    """
    Run a black-box attack on a model for a specified number of batches.
    
    Args:
        model: The model to attack.
        test_loader: DataLoader for the test dataset.
        n_batches: Number of batches to process.
        
    Returns:
        success_rate: Success rate of the attack.
        avg_perturbation: Average perturbation of the adversarial examples.
        avg_queries: Average number of queries made (if applicable).
    """
    # Ensure the model is in evaluation mode
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    
    # Initialize lists to store results
    successes, perturbations, queries = [], [], []

    # Iterate over three batches of images
    for i in tqdm(range(n_batches), desc="Attacking model"):
        images, labels = next(iter(test_loader))
        images, labels = images.to(device), labels.to(device)
        # Optionally, filter out misclassified samples
        preds = model(images).argmax(dim=1)
        mask = preds == labels
        if mask.sum() == 0:
            continue
        images, labels = images[mask], labels[mask]
        # Run attack
        attack = fb.attacks.BoundaryAttack()
        raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=None)
        # Collect metrics
        successes.append(success.cpu())
        perturbations.append((clipped_advs - images).view(images.size(0), -1).norm(dim=1).cpu())
        # If available, track queries (depends on attack implementation)
        queries.append(attack.queries.cpu()) if hasattr(attack, 'queries') else None
    # Aggregate and report
    if successes:
        success_rate = torch.cat(successes).float().mean().item()
        avg_perturbation = torch.cat(perturbations).mean().item()
        avg_queries = torch.cat(queries).float().mean().item() if queries else None
    else:
        print("No successful batches processed. Check your data or model accuracy.")
        success_rate, avg_perturbation, avg_queries = None, None, None
    return success_rate, avg_perturbation, avg_queries

# extract the learning rate, beta_1, and beta_2 from the model name
def extract_params(model_name):
    parts = model_name.split('_')
    learning_rate = float(parts[2])
    beta_1 = float(parts[4])
    beta_2 = float(parts[6][:-4])
    return learning_rate, beta_1, beta_2

In [48]:
set_seed(42)

n_batches = 3
batch_size = 1
_, _, test_loader = get_data_loaders(batch_size=batch_size)

drive_base_path = os.getcwd()
folder_path = os.path.join(drive_base_path, 'Results/Adam_VGG')
csv_path = os.path.join(folder_path, 'VGGBlackBox.csv')
if not os.path.exists(csv_path):
    columns = ["file_name", "learning_rate", "beta_1", "beta_2","success_rate","avg_perturbations","avg_queries"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)

i = 0

for model_name in os.listdir(folder_path):
    print("Loading model: ", model_name)
    i += 1
    if not model_name.endswith('.pth'):
        continue
    model_path = os.path.join(folder_path, model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = VGGLike().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    results = attack_n_batches(model, test_loader, device, n_batches)
    print("Saving results")
    df = pd.DataFrame([model_name] + list(extract_params(model_name)) + list(results)).T
    df.to_csv(csv_path, mode='a', header=False, index=False)
    clear_output(wait=True)
    

Loading model:  VGG_lr_5e-05_beta1_0.9_beta2_0.999.pth


Attacking model: 100%|██████████| 3/3 [04:51<00:00, 97.11s/it] 

Saving results
